In [6]:
from flask import Flask, render_template, request
from tensorflow import keras
import numpy as np
import jsonify
import json

from konlpy.tag import Okt
okt = Okt()

In [15]:
model = keras.models.load_model('data/predict_genre_by_lstm.h5')

lyric = '내 랩을 들어 나는 우리나라 최고의 래퍼다'

# Load the token-to-index dictionary
with open('data/token_to_index.json') as f:
    index_dict = json.load(f)['0']
    input_tokenized = okt.nouns(lyric)

    input_indexed = []

    # Convert the tokens to their index values
    for t in input_tokenized:
        if t in index_dict:
           input_indexed.append(index_dict[t])
    # Pad the sequence to the right length
    input_indexed = np.array([input_indexed])
    # pad to 105
    input_padded = keras.preprocessing.sequence.pad_sequences(input_indexed, maxlen=105, padding='post')

    pred = model.predict(input_padded)
    genre_list = ['발라드', '댄스', '힙합', '트로트']

    # Get the index of the highest probability
    genre = genre_list[np.argmax(pred)]




1/1 [==============================] - 1s 1s/step
힙합


In [ ]:
app = Flask(__name__)

@app.route('/predictgenrebylstm', mothods=['POST'])
def predict_genre_by_dl():
    # parameter로 넘어온 lyric을 받습니다.
    lyric = request.args.get('lyric')

    # Null Check와 Empyt 체크를 합니다.
    if lyric is None or lyric == '':
        return jsonify({'result': '가사가 입력되지 않았습니다. - No parameter'})

    else:
        # 저장한 모델을 로드합니다.
        model = keras.models.load_model('data/predict_genre_by_lstm.h5')

        # Load the token-to-index dictionary
        with open('data/token_to_index.json') as f:
            tokenizer = json.load(f)

        return label